In [30]:
import pandas as pd

# Replace 'stocks_data.csv' with the actual path to your CSV file
file_path = 'ind_niftysmallcap250list.csv'

def extract_stock_codes(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Extract the "Stock Code" column into a list
        stock_codes_list = df['Symbol'].astype(str).tolist()

        return stock_codes_list

    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage:
stock_codes_list = extract_stock_codes(file_path)


import yfinance as yf

def get_stock_data_list(stock_codes, period='1y'):
    all_stock_data = []

    for stock_code in stock_codes:
        try:
            yahoo_ticker = stock_code + '.NS'
            stock = yf.Ticker(yahoo_ticker)
            data = stock.history(period=period)
            if not data.empty:
                # Extract relevant features from historical stock data
                stock_data = {
                    'Date': data.index.tolist(),
                    'Ticker': stock_code,
                    'Open': data['Open'].tolist(),
                    'High': data['High'].tolist(),
                    'Low': data['Low'].tolist(),
                    'Close': data['Close'].tolist(),
                    'Volume': data['Volume'].tolist(),
                    'Sector': stock.info.get('industry', 'N/A')
                }
                all_stock_data.append(pd.DataFrame(stock_data))
            else:
                print(f"No data available for {stock_code}")
        except Exception as e:
            print(f"Error fetching data for {stock_code}: {e}")

    return pd.concat(all_stock_data, ignore_index=True)

# Example usage:
duration_period = '50d'  # Change the duration as needed, e.g., '1mo', '3mo', '1y', etc.

stock_data_df = get_stock_data_list(stock_codes_list, period=duration_period)

# Display the DataFrame
stock_data_df


,Date,Ticker,Open,High,Low,Close,Volume,Sector
0,2023-12-06 00:00:00+05:30,360ONE,610.608542,653.102913,605.445661,644.812561,2168523,Asset Management
1,2023-12-07 00:00:00+05:30,360ONE,645.358648,645.358648,632.997535,636.422913,719592,Asset Management
2,2023-12-08 00:00:00+05:30,360ONE,640.195723,647.344323,635.430010,642.926086,538465,Asset Management
3,2023-12-11 00:00:00+05:30,360ONE,642.926096,647.344333,627.238930,641.883606,203623,Asset Management
4,2023-12-12 00:00:00+05:30,360ONE,637.465380,644.862195,623.366764,639.848267,290724,Asset Management
...,...,...,...,...,...,...,...,...
12491,2024-02-12 00:00:00+05:30,ECLERX,2477.000000,2545.000000,2381.000000,2399.500000,51173,Information Technology Services
12492,2024-02-13 00:00:00+05:30,ECLERX,2383.000000,2406.250000,2364.149902,2395.199951,52591,Information Technology Services
12493,2024-02-14 00:00:00+05:30,ECLERX,2363.300049,2398.850098,2352.000000,2363.850098,50353,Information Technology Services
12494,2024-02-15 00:00:00+05:30,ECLERX,2376.000000,2384.199951,2305.050049,2318.050049,77476,Information Technology Services


In [58]:
# Group by Ticker and calculate the requested statistics
ticker_grouped = stock_data_df.groupby('Ticker')
agg_functions = {
    'Close': ['min', 'max', 'median', 'mean', 'var', 'std']
}

stock_summary = ticker_grouped.agg(agg_functions).reset_index()

# Flatten the multi-level columns
stock_summary.columns = ['{}_{}'.format(col[0], col[1]) for col in stock_summary.columns]
stock_summary = stock_summary.rename(columns={'Ticker_': 'Ticker'})

# Display the summary DataFrame
stock_summary


,Ticker,Close_min,Close_max,Close_median,Close_mean,Close_var,Close_std
0,360ONE,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674
1,AARTIDRUGS,471.941620,538.650024,509.020264,510.240392,376.220089,19.396394
2,AAVAS,1385.449951,1613.000000,1503.149963,1507.198994,3008.668363,54.851330
3,ACI,588.445862,660.485168,626.961700,628.312844,254.213863,15.944086
4,AEGISCHEM,333.950012,436.200012,372.150009,372.656001,396.296526,19.907198
...,...,...,...,...,...,...,...
245,WELCORP,515.049988,599.849976,553.450012,557.427996,546.363270,23.374415
246,WELSPUNLIV,135.949997,163.800003,152.349998,150.930999,34.097240,5.839284
247,WESTLIFE,776.950012,866.750000,824.399994,827.748000,386.333703,19.655373
248,ZENSARTECH,516.819275,625.035889,568.261993,568.227913,909.130320,30.151788


In [70]:
# Left join the ticker_features DataFrame with the original stock_data_df DataFrame
merged_df = pd.merge(stock_data_df, stock_summary, on='Ticker', how='left')

merged_df['Close_z_score'] = (merged_df['Close'] - merged_df['Close_mean']) / merged_df['Close_std']

# Format the 'Date' column
merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m-%d')
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Define the conditions and corresponding values for the new column
conditions = [
    (merged_df['Close_z_score'] <= 1) & (merged_df['Close_z_score'] >= -1),
    (merged_df['Close_z_score'] <= 2) & (merged_df['Close_z_score'] >= -2),
    (merged_df['Close_z_score'] <= 3) & (merged_df['Close_z_score'] >= -3),
    (merged_df['Close_z_score'] > 3) & (merged_df['Close_z_score'] < -3),
]

# Define corresponding values for each condition
values = ['Within_1', 'Within_2', 'Within_3', 'More_3']

# Create the new column based on the conditions
merged_df['ZScore_Category'] = np.select(conditions, values, default='Other')


# Sort the DataFrame by 'Ticker' and 'Date'
merged_df = merged_df.sort_values(by=['Ticker', 'Date'], ascending=[True, False]).reset_index(drop=True)

# Display the merged DataFrame
merged_df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Sector,Close_min,Close_max,Close_median,Close_mean,Close_var,Close_std,Close_z_score,ZScore_Category
0,2024-02-16,360ONE,713.049988,715.000000,685.049988,692.400024,930453,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.382943,Within_2
1,2024-02-15,360ONE,712.000000,726.950012,705.700012,713.599976,665907,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,2.153692,Within_3
2,2024-02-14,360ONE,690.250000,719.950012,680.650024,707.849976,696379,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.944644,Within_2
3,2024-02-13,360ONE,650.750000,698.000000,647.049988,691.700012,1283704,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.357494,Within_2
4,2024-02-12,360ONE,651.849976,666.000000,641.049988,648.250000,1146649,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,-0.222181,Within_1


In [78]:
# Subset only one row for each Ticker with Max Date
max_date_indices = merged_df.groupby('Ticker')['Date'].idxmax()
subset_df = merged_df.loc[max_date_indices]
subset_df = subset_df.reset_index(drop=True)
subset_df.shape
subset_df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Sector,Close_min,Close_max,Close_median,Close_mean,Close_var,Close_std,Close_z_score,ZScore_Category
0,2024-02-16,360ONE,713.049988,715.000000,685.049988,692.400024,930453,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.382943,Within_2
1,2024-02-16,AARTIDRUGS,536.000000,545.500000,526.599976,529.950012,142173,Drug Manufacturers - Specialty & Generic,471.941620,538.650024,509.020264,510.240392,376.220089,19.396394,1.016149,Within_2
2,2024-02-16,AAVAS,1426.000000,1426.599976,1403.300049,1422.750000,334036,Mortgage Finance,1385.449951,1613.000000,1503.149963,1507.198994,3008.668363,54.851330,-1.539598,Within_2
3,2024-02-16,ACI,634.650024,664.299988,628.099976,659.150024,722138,Chemicals,588.445862,660.485168,626.961700,628.312844,254.213863,15.944086,1.934083,Within_2
4,2024-02-16,AEGISCHEM,441.000000,468.649994,429.000000,436.200012,7699243,Oil & Gas Refining & Marketing,333.950012,436.200012,372.150009,372.656001,396.296526,19.907198,3.192012,Other


In [79]:
subset_df[['Ticker','ZScore_Category']].value_counts().T

Ticker      ZScore_Category
360ONE      Within_2           1
NSLNISP     Within_2           1
MMTC        Within_1           1
MOTILALOFS  Within_2           1
MRPL        Within_3           1
                              ..
GOCOLORS    Within_1           1
GODFRYPHLP  Within_2           1
GPIL        Within_1           1
GPPL        Within_3           1
ZYDUSWELL   Within_1           1
Name: count, Length: 250, dtype: int64

## RSI stands for Relative Strength Index

## RSI values range from 0 to 100.
## Generally, an RSI above 70 indicates overbought conditions, suggesting a potential reversal or correction.
## An RSI below 30 indicates oversold conditions, suggesting a potential upward reversal.

In [71]:
# Define a function to calculate RSI
def calculate_rsi(data, column='Close', period=14):
    # Calculate daily price changes
    delta = data[column].diff(1)

    # Calculate gains (positive changes) and losses (negative changes)
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)

    # Calculate average gains and losses over the specified period
    avg_gains = gains.rolling(window=period, min_periods=1).mean()
    avg_losses = losses.rolling(window=period, min_periods=1).mean()

    # Calculate relative strength (RS)
    rs = avg_gains / avg_losses

    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [84]:
# Calculate RSI and add it to the DataFrame
stock_data_df = merged_df[merged_df.Ticker=='360ONE']
stock_data_df['RSI'] = calculate_rsi(stock_data_df)
stock_data_df = pd.DataFrame(stock_data_df)
stock_data_df.head()

C:\Users\Abhishek Jaiswal\AppData\Local\Temp\ipykernel_11188\585140135.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data_df['RSI'] = calculate_rsi(stock_data_df)


,Date,Ticker,Open,High,Low,Close,Volume,Sector,Close_min,Close_max,Close_median,Close_mean,Close_var,Close_std,Close_z_score,ZScore_Category,RSI
0,2024-02-16,360ONE,713.049988,715.000000,685.049988,692.400024,930453,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.382943,Within_2,NaN
1,2024-02-15,360ONE,712.000000,726.950012,705.700012,713.599976,665907,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,2.153692,Within_3,100.000000
2,2024-02-14,360ONE,690.250000,719.950012,680.650024,707.849976,696379,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.944644,Within_2,78.664154
3,2024-02-13,360ONE,650.750000,698.000000,647.049988,691.700012,1283704,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,1.357494,Within_2,49.187919
4,2024-02-12,360ONE,651.849976,666.000000,641.049988,648.250000,1146649,Asset Management,604.950012,713.599976,649.876129,654.361232,756.562104,27.505674,-0.222181,Within_1,24.494476
